In [5]:
 #Stage 2: ETL Pipeline with NLP Preprocessing
# ==============================================

# Install NLP libraries
!pip install pandas numpy nltk  scikit-learn 
!python -m spacy download en_core_web_sm

import pandas as pd
import numpy as np
import json
import os
import sqlite3
import re
from pathlib import Path

Defaulting to user installation because normal site-packages is not writeable
  Using cached nltk-3.9.2-py3-none-any.whl.metadata (3.2 kB)
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.5 MB 2.7 MB/s eta 0:00:01
   --------------------------- ------------ 1.0/1.5 MB 3.0 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 2.9 MB/s  0:00:00

   ---------------------------------------- 0/3 [regex]
   ---------------------------------------- 0/3 [regex]
   ------------- -------------------------- 1/3 [click]
   ------------- -------------------------- 1/3 [click]
   ------------- -------------------------- 1/3 [click]
   -------------------------- ------------- 2/3 [nltk]
   -------------------------- ------------- 2/3 [nltk]
   -------------------------- ------------- 2/3 [nltk]
   -------------------------- ------------- 2/3 [nltk]
   -------------------------- ------------- 2/3 [nltk]
   --------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\Hp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
C:\Users\Hp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe: No module named spacy


In [7]:
!pip install spacy
!python -m spacy download en_core_web_sm


Defaulting to user installation because normal site-packages is not writeable
  Using cached spacy-3.8.11-cp313-cp313-win_amd64.whl.metadata (28 kB)
   ---------------------------------------- 0.0/14.2 MB ? eta -:--:--
   - -------------------------------------- 0.5/14.2 MB 3.0 MB/s eta 0:00:05
   -- ------------------------------------- 1.0/14.2 MB 2.9 MB/s eta 0:00:05
   ---- ----------------------------------- 1.6/14.2 MB 2.9 MB/s eta 0:00:05
   ------ --------------------------------- 2.4/14.2 MB 3.0 MB/s eta 0:00:04
   -------- ------------------------------- 2.9/14.2 MB 3.0 MB/s eta 0:00:04
   ---------- ----------------------------- 3.7/14.2 MB 3.0 MB/s eta 0:00:04
   ----------- ---------------------------- 4.2/14.2 MB 2.9 MB/s eta 0:00:04
   ------------- -------------------------- 4.7/14.2 MB 2.9 MB/s eta 0:00:04
   --------------- ------------------------ 5.5/14.2 MB 2.9 MB/s eta 0:00:03
   ---------------- ----------------------- 6.0/14.2 MB 2.9 MB/s eta 0:00:03
   --------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\Hp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     - -------------------------------------- 0.5/12.8 MB 2.9 MB/s eta 0:00:05
     --- ------------------------------------ 1.0/12.8 MB 3.0 MB/s eta 0:00:04
     ---- ----------------------------------- 1.6/12.8 MB 2.6 MB/s eta 0:00:05
     ------ --------------------------------- 2.1/12.8 MB 2.6 MB/s eta 0:00:05
     -------- ------------------------------- 2.6/12.8 MB 2.6 MB/s eta 0:00:04
     --------- ------------------------------ 3.1/12.8 MB 2.6 MB/s eta 0:00:04
     ----------- ---------------------------- 3.7/12.8 MB 2.6 MB/s eta 0:00:04
     ------------- -------------------------- 4.2/12.8 MB 2.6 MB/s eta 0:00:04
     --------------- ------------------------ 5.0/12.8 MB 2.7 MB/s eta 0:00:03
     ----------------- ---------------------- 5.5/12.8 MB 2.6 MB/s eta 0:00:03
     ------------------ --------------------- 6.0/12.8 MB 2.


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\Hp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
# NLP libraries
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import spacy

In [9]:
# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...


True

In [10]:
# Load spaCy
nlp = spacy.load('en_core_web_sm')

In [11]:
# ===========================
# LOAD PATHS
# ===========================
with open('data/paths_config.json', 'r') as f:
    paths = json.load(f)

print("📂 Loaded paths configuration")

📂 Loaded paths configuration


In [12]:
# ===========================
# EXTRACT: Load Raw Data
# ===========================
print("\n" + "="*60)
print("STAGE 1: EXTRACT - Loading Raw Data")
print("="*60)


STAGE 1: EXTRACT - Loading Raw Data


In [13]:
# Load Sentiment Data (Amazon Reviews)
def load_fasttext_format(filepath):
    """Load FastText format: __label__X text"""
    texts = []
    labels = []
    
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line.startswith('__label__'):
                # Extract label and text
                parts = line.split(' ', 1)
                label = parts[0].replace('__label__', '')
                text = parts[1] if len(parts) > 1 else ''
                
                labels.append(int(label))  # 1 or 2
                texts.append(text)
    
    return pd.DataFrame({'text': texts, 'sentiment': labels})

In [16]:
import bz2

In [23]:
# Try loading sentiment data
# 1. Load Sentiment Data (Handling the BZ2 format)
sentiment_file_bz2 = os.path.join(paths['sentiment_path'], 'train.ft.txt.bz2')

if os.path.exists(sentiment_file_bz2):
    print("📦 Loading compressed Sentiment data...")
    # Use the logic we built: stream from bz2, split at first space
    data = []
    with bz2.open(sentiment_file_bz2, "rt", encoding="utf-8") as f:
        for i, line in enumerate(f):
            if i >= 10000: break  # Sampling 10k
            parts = line.split(" ", 1)
            if len(parts) > 1:
                label = 1 if parts[0] == "__label__2" else 0 # 1 for positive
                data.append({"text": parts[1].strip(), "label": label})
    sentiment_df = pd.DataFrame(data)
    sentiment_df['sentiment'] = sentiment_df['label'].map({0: 'negative', 1: 'positive'})
else:
    print("⚠️ BZ2 file not found. Check paths.")

# 2. Load Intent Data (Fixing the ParserError)
#2. Load Intent Data (Correctly as JSON)
intent_file = os.path.join(paths['intent_path'], paths['intent_files'][0])

try:
    with open(intent_file, 'r', encoding='utf-8') as f:
        raw_json = json.load(f)
    
    # Flatten the JSON (Intents -> Text patterns)
    formatted_data = []
    for item in raw_json['intents']:
        tag = item['intent']
        for pattern in item['text']:
            formatted_data.append({'intent': tag, 'text': pattern})
    
    intent_df = pd.DataFrame(formatted_data)
    print(f"✅ Loaded {len(intent_df)} intent samples from JSON")

except Exception as e:
    print(f"❌ Intent load failed: {e}")

📦 Loading compressed Sentiment data...
✅ Loaded 143 intent samples from JSON


In [24]:
try:
    # This is the specific one the error asked for
    nltk.download('punkt_tab')
    # These are usually needed for your Preprocessor class
    nltk.download('stopwords')
    nltk.download('wordnet')
    print("✅ NLTK Resources Downloaded!")
except Exception as e:
    print(f"❌ Download failed: {e}")

✅ NLTK Resources Downloaded!


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [25]:
# ===========================
# TRANSFORM: NLP Preprocessing
# ===========================
print("\n" + "="*60)
print("STAGE 2: TRANSFORM - NLP Preprocessing")
print("="*60)

class TextPreprocessor:
    """Complete NLP preprocessing pipeline"""
    
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
    
    def clean_text(self, text):
        """Basic cleaning"""
        if pd.isna(text):
            return ""
        
        # Convert to lowercase
        text = str(text).lower()
        
        # Remove URLs
        text = re.sub(r'http\S+|www\S+|https\S+', '', text)
        
        # Remove email addresses
        text = re.sub(r'\S+@\S+', '', text)
        
        # Remove special characters but keep spaces
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
        
        # Remove extra whitespace
        text = ' '.join(text.split())
        
        return text
    
    def remove_stopwords(self, text):
        """Remove common stopwords"""
        tokens = word_tokenize(text)
        filtered = [w for w in tokens if w not in self.stop_words]
        return ' '.join(filtered)
    
    def lemmatize(self, text):
        """Lemmatize words to base form"""
        tokens = word_tokenize(text)
        lemmatized = [self.lemmatizer.lemmatize(w) for w in tokens]
        return ' '.join(lemmatized)
    
    def preprocess(self, text, remove_stops=True, lemmatize=True):
        """Complete preprocessing pipeline"""
        # Clean
        text = self.clean_text(text)
        
        # Remove stopwords
        if remove_stops:
            text = self.remove_stopwords(text)
        
        # Lemmatize
        if lemmatize:
            text = self.lemmatize(text)
        
        return text

# Initialize preprocessor
preprocessor = TextPreprocessor()

# Preprocess sentiment data
print("🔄 Processing sentiment data...")
sentiment_df['text_original'] = sentiment_df['text']
sentiment_df['text_cleaned'] = sentiment_df['text'].apply(preprocessor.preprocess)
sentiment_df['text_length'] = sentiment_df['text_cleaned'].str.len()

# Preprocess intent data (adjust column name as needed)
print("🔄 Processing intent data...")
# Let's see what we actually have
print(f"Available columns: {intent_df.columns.tolist()}")

# Ensure 'text' exists
if 'text' in intent_df.columns:
    intent_df['text_original'] = intent_df['text']
    intent_df['text_cleaned'] = intent_df['text'].apply(preprocessor.preprocess)
    intent_df['text_length'] = intent_df['text_cleaned'].str.len()
else:
    # If it's not named 'text', take the very first column available
    first_col = intent_df.columns[0]
    intent_df['text_original'] = intent_df[first_col]
    intent_df['text_cleaned'] = intent_df[first_col].apply(preprocessor.preprocess)
    
# Safely handle the 'intent' column
if 'intent' in intent_df.columns:
    # It's already there, no need to do anything
    pass
else:
    print("⚠️ Warning: 'intent' column not found. Check your JSON flattening step.")

print("✅ Intent processing complete!")


STAGE 2: TRANSFORM - NLP Preprocessing
🔄 Processing sentiment data...
🔄 Processing intent data...
Available columns: ['intent', 'text']
✅ Intent processing complete!


In [26]:
# ===========================
# Data Quality Checks
# ===========================
print("\n📊 Data Quality Metrics:")
print(f"Sentiment - Empty texts: {(sentiment_df['text_cleaned'] == '').sum()}")
print(f"Intent - Empty texts: {(intent_df['text_cleaned'] == '').sum()}")

# Remove empty texts
sentiment_df = sentiment_df[sentiment_df['text_cleaned'].str.len() > 0]
intent_df = intent_df[intent_df['text_cleaned'].str.len() > 0]

print(f"After cleaning - Sentiment: {len(sentiment_df)}, Intent: {len(intent_df)}")



📊 Data Quality Metrics:
Sentiment - Empty texts: 0
Intent - Empty texts: 5
After cleaning - Sentiment: 10000, Intent: 138


In [27]:
# ===========================
# LOAD: Store in SQLite
# ===========================
print("\n" + "="*60)
print("STAGE 3: LOAD - Storing in SQLite Warehouse")
print("="*60)



STAGE 3: LOAD - Storing in SQLite Warehouse


In [28]:
# Create SQLite database
db_path = 'data/support_warehouse.db'
conn = sqlite3.connect(db_path)

# Store sentiment data
sentiment_df[['text_original', 'text_cleaned', 'sentiment', 'text_length']].to_sql(
    'sentiment_data', 
    conn, 
    if_exists='replace', 
    index=False
)
print(f"✅ Stored {len(sentiment_df)} records in 'sentiment_data' table")


✅ Stored 10000 records in 'sentiment_data' table


In [29]:
# Store intent data
intent_df[['text_original', 'text_cleaned', 'intent', 'text_length']].to_sql(
    'intent_data',
    conn,
    if_exists='replace',
    index=False
)
print(f"✅ Stored {len(intent_df)} records in 'intent_data' table")

✅ Stored 138 records in 'intent_data' table


In [30]:
# Create combined table for multi-task learning
print("\n🔗 Creating combined dataset for multi-task model...")

# Add source column
sentiment_df['source'] = 'sentiment'
intent_df['source'] = 'intent'

# Combine (for multi-task, we'll use separate heads)
combined_df = pd.DataFrame({
    'text_original': list(sentiment_df['text_original']) + list(intent_df['text_original']),
    'text_cleaned': list(sentiment_df['text_cleaned']) + list(intent_df['text_cleaned']),
    'sentiment': list(sentiment_df['sentiment']) + [None] * len(intent_df),
    'intent': [None] * len(sentiment_df) + list(intent_df['intent']),
    'source': list(sentiment_df['source']) + list(intent_df['source'])
})

combined_df.to_sql('combined_data', conn, if_exists='replace', index=False)
print(f"✅ Stored {len(combined_df)} records in 'combined_data' table")



🔗 Creating combined dataset for multi-task model...
✅ Stored 10138 records in 'combined_data' table


In [31]:
# Create metadata table
metadata = {
    'table_name': ['sentiment_data', 'intent_data', 'combined_data'],
    'record_count': [len(sentiment_df), len(intent_df), len(combined_df)],
    'created_date': [pd.Timestamp.now()] * 3
}
pd.DataFrame(metadata).to_sql('metadata', conn, if_exists='replace', index=False)

conn.close()
print(f"\n✅ ETL Pipeline Complete! Database saved at: {db_path}")



✅ ETL Pipeline Complete! Database saved at: data/support_warehouse.db


In [32]:
# Save processed data as CSV for model training
sentiment_df.to_csv('data/processed/sentiment_processed.csv', index=False)
intent_df.to_csv('data/processed/intent_processed.csv', index=False)
combined_df.to_csv('data/processed/combined_processed.csv', index=False)

print("\n📝 Next Step: Run 3_model_training.ipynb")


📝 Next Step: Run 3_model_training.ipynb
